Downloading Sentinel-2 GlobalMosaic Data:
=========================================

Links:
- https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Process/Examples/S2L2A.html#true-color
- https://documentation.dataspace.copernicus.eu/notebook-samples/sentinelhub/cloudless_process_api.html



In [35]:
# Print basic timestamp and python version information

import time
import sys

print("Timestamp: ", time.strftime("%Y-%m-%d %H:%M:%S"))
print("Python version: ", sys.version)

Timestamp:  2025-02-05 14:45:54
Python version:  3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [2]:
# Update pip itself
!pip install --upgrade pip

# Setup the environment with pip installs and imports
installs = ["sentinelhub", "numpy", "pandas", "matplotlib", "requests", "getpass", "scipy"]
for package in installs:
    !pip install {package}

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass


In [3]:
# Utilities
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import getpass

from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)


In [4]:
# Your client credentials
client_id     = 'sh-d89408a3-f5db-4c8b-a39d-929f9f299aa9'
client_secret = 'BqJ60XbDC85suN5gVlqYy6JjmJ5nwdT5'

In [23]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

def SetupOAuth(client_id, client_secret):

    # Create a session
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)

    # Get token for the session
    token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                            client_secret=client_secret, include_client_id=True)

    # All requests using this session will have an access token automatically added
    resp = oauth.get("https://sh.dataspace.copernicus.eu/configuration/v1/wms/instances")

    return oauth

oauth = SetupOAuth(client_id, client_secret)


In [6]:
# Create strings to support the query

def CreateBBoxString(minlon, minlat, maxlon, maxlat):
    return f"[{minlon}, {minlat}, {maxlon}, {maxlat}]"

def CreateTimerangeString(date):
    # add buffer around the stated date
    datespace = datetime.timedelta(days=2)
    startdate = date - datespace
    enddate = date + datespace

    # format individual date strings
    from_date = f"{startdate.strftime('%Y-%m-%d')}T00:00:00Z"
    to_date = f"{enddate.strftime('%Y-%m-%d')}T23:59:59Z"

    # return the collated/formatted answer
    return {"from": from_date, "to": to_date}



In [7]:
globalmosaic_dates = {
    "2023Q1": "2023-01-01",
    "2023Q2": "2023-04-01",
    "2023Q3": "2023-07-01",
    "2023Q4": "2023-10-01",
    "2024Q1": "2024-01-01",
    "2024Q2": "2024-04-01",
    "2024Q3": "2024-07-01",
}

In [8]:
from PIL import Image
from io import BytesIO


def download_GM2(outpath, datestr, bl_lat, bl_lon, width):

    oauth = SetupOAuth(client_id, client_secret)

    minlat = bl_lat
    maxlat = bl_lat + width
    minlon = bl_lon
    maxlon = bl_lon + width

    # datestr = "2023-07-01"
    fromstr = f"{datestr}T00:00:00Z"
    tostr   = f"{datestr}T23:59:59Z"

    evalscript_GM = """
    //VERSION=3
    function setup() {
        return {
            input: ["B02", "B03", "B04"],
            output: { bands: 3 }
        };
    }

    function evaluatePixel(sample) {
        return [3.5 * sample.B04/10000, 3.5 * sample.B03/10000, 3.5 * sample.B02/10000];
    }
    """

    request_GM = {
    "input": {
        "bounds": {
        "bbox": [minlon, minlat, maxlon, maxlat]
        },
        "data": [
        {
            "dataFilter": {
            "timeRange": {
                "from": fromstr,
                "to": tostr
            }
            },
            "type": "byoc-5460de54-082e-473a-b6ea-d5cbe3c17cca"

        }
        ]
    },
    "output": {
        "width": 1024,
        "height": 1024,
        "responses": [
        {
            "identifier": "default",
            "format": {
                "type": "image/png"
            }
        }
        ]
    },
    "evalscript": evalscript_GM,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request_GM)

    # Output the result to a file
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        image.save(outpath)
    else:
        print(f"Request failed with status code {response.status_code}")



In [9]:
import os

# function to make sure a path exists. If not, create it
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [10]:
def create_5deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (4 - currlatid)

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + currlongid

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.png"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 1  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [11]:
def create_1deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    latdelta  = 0.2
    longdelta = 0.2

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (1 - ((currlatid + 1) * latdelta))

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + (currlongid * longdelta)

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.png"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 0.2  # Assuming each 1-degree tile has a width of 1
            }

    return dictionary

In [31]:
def create_0p2deg_dictionary(path, filenamestart, startlat, startlong):
    dictionary = {}

    latdelta  = 0.04
    longdelta = 0.04

    # Loop over 5 lats
    for currlatid in range(5):
        currlatletter = chr(ord('A') + currlatid)  # Get letter A-E for lat
        currlat = startlat + (0.2 - ((currlatid + 1) * latdelta))

        # Loop over 5 longs
        for currlongid in range(5):
            currlongletter = chr(ord('A') + currlongid)  # Get letter A-E for long
            currlong = startlong + (currlongid * longdelta)

            # Generate the filename based on letters
            currfilename = f"{filenamestart}{currlatletter}{currlongletter}.png"

            # Create dictionary entry
            dictionary[currfilename] = {
                "localpath": path,
                "filename": currfilename,
                "lat": currlat,
                "long": currlong,
                "width": 0.04
            }

    return dictionary

In [12]:
temp_data = create_1deg_dictionary("Lvl2_1x1/BF_BF_CA/", "Sat_BF_BF_CA_", 52, -5)

# print temp_data verbose/formatted
for key, value in temp_data.items():
    print(f"\"{key}\": {value},")

"Sat_BF_BF_CA_AA.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AA.png', 'lat': 52.8, 'long': -5.0, 'width': 0.2},
"Sat_BF_BF_CA_AB.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AB.png', 'lat': 52.8, 'long': -4.8, 'width': 0.2},
"Sat_BF_BF_CA_AC.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AC.png', 'lat': 52.8, 'long': -4.6, 'width': 0.2},
"Sat_BF_BF_CA_AD.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AD.png', 'lat': 52.8, 'long': -4.4, 'width': 0.2},
"Sat_BF_BF_CA_AE.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_AE.png', 'lat': 52.8, 'long': -4.2, 'width': 0.2},
"Sat_BF_BF_CA_BA.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_BA.png', 'lat': 52.6, 'long': -5.0, 'width': 0.2},
"Sat_BF_BF_CA_BB.png": {'localpath': 'Lvl2_1x1/BF_BF_CA/', 'filename': 'Sat_BF_BF_CA_BB.png', 'lat': 52.6, 'long': -4.8, 'width': 0.2},
"Sat_BF_BF_CA_BC.png": {'localpath': 'Lvl2_1x1/B

In [24]:
# BB - North America - West Coast

BB_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AA/", "Sat_BB_AA_", 55, -150)
BB_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AB/", "Sat_BB_AB_", 55, -145)
BB_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AC/", "Sat_BB_AC_", 55, -140)
BB_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AD/", "Sat_BB_AD_", 55, -135)
BB_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AE/", "Sat_BB_AE_", 55, -130)
BB_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_AF/", "Sat_BB_AF_", 55, -125)

BB_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BA/", "Sat_BB_BA_", 50, -150)
BB_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BB/", "Sat_BB_BB_", 50, -145)
BB_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BC/", "Sat_BB_BC_", 50, -140)
BB_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BD/", "Sat_BB_BD_", 50, -135)
BB_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BE/", "Sat_BB_BE_", 50, -130)
BB_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_BF/", "Sat_BB_BF_", 50, -125)

BB_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CA/", "Sat_BB_CA_", 45, -150)
BB_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CB/", "Sat_BB_CB_", 45, -145)
BB_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CC/", "Sat_BB_CC_", 45, -140)
BB_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CD/", "Sat_BB_CD_", 45, -135)
BB_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CE/", "Sat_BB_CE_", 45, -130)
BB_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_CF/", "Sat_BB_CF_", 45, -125)

BB_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DA/", "Sat_BB_DA_", 40, -150)
BB_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DB/", "Sat_BB_DB_", 40, -145)
BB_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DC/", "Sat_BB_DC_", 40, -140)
BB_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DD/", "Sat_BB_DD_", 40, -135)
BB_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DE/", "Sat_BB_DE_", 40, -130)
BB_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_DF/", "Sat_BB_DF_", 40, -125)

BB_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_EA/", "Sat_BB_EA_", 35, -150)
BB_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_EB/", "Sat_BB_EB_", 35, -145)
BB_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_EC/", "Sat_BB_EC_", 35, -140)
BB_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_ED/", "Sat_BB_ED_", 35, -135)
BB_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_EE/", "Sat_BB_EE_", 35, -130)
BB_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_EF/", "Sat_BB_EF_", 35, -125)

BB_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FA/", "Sat_BB_FA_", 30, -150)
BB_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FB/", "Sat_BB_FB_", 30, -145)
BB_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FC/", "Sat_BB_FC_", 30, -140)
BB_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FD/", "Sat_BB_FD_", 30, -135)
BB_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FE/", "Sat_BB_FE_", 30, -130)
BB_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BB_FF/", "Sat_BB_FF_", 30, -125)

BB_1Degs_List = [
    BB_AA_1Deg, BB_AB_1Deg, BB_AC_1Deg, BB_AD_1Deg, BB_AE_1Deg, BB_AF_1Deg,
                                        BB_BD_1Deg, BB_BE_1Deg, BB_BF_1Deg,
                                                    BB_CE_1Deg, BB_CF_1Deg,
                                                                BB_DF_1Deg,
                                                                BB_EF_1Deg,
                                                                BB_FF_1Deg,
]

In [65]:
# BC - North America - Central

BC_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AA/", "Sat_BC_AA_", 55, -120)
BC_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AB/", "Sat_BC_AB_", 55, -115)
BC_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AC/", "Sat_BC_AC_", 55, -110)
BC_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AD/", "Sat_BC_AD_", 55, -105)
BC_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AE/", "Sat_BC_AE_", 55, -100)
BC_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_AF/", "Sat_BC_AF_", 55,  -95)

BC_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BA/", "Sat_BC_BA_", 50, -120)
BC_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BB/", "Sat_BC_BB_", 50, -115)
BC_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BC/", "Sat_BC_BC_", 50, -110)
BC_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BD/", "Sat_BC_BD_", 50, -105)
BC_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BE/", "Sat_BC_BE_", 50, -100)
BC_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_BF/", "Sat_BC_BF_", 50,  -95)

BC_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CA/", "Sat_BC_CA_", 45, -120)
BC_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CB/", "Sat_BC_CB_", 45, -115)
BC_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CC/", "Sat_BC_CC_", 45, -110)
BC_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CD/", "Sat_BC_CD_", 45, -105)
BC_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CE/", "Sat_BC_CE_", 45, -100)
BC_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_CF/", "Sat_BC_CF_", 45,  -95)

BC_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DA/", "Sat_BC_DA_", 40, -120)
BC_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DB/", "Sat_BC_DB_", 40, -115)
BC_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DC/", "Sat_BC_DC_", 40, -110)
BC_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DD/", "Sat_BC_DD_", 40, -105)
BC_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DE/", "Sat_BC_DE_", 40, -100)
BC_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_DF/", "Sat_BC_DF_", 40,  -95)

BC_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_EA/", "Sat_BC_EA_", 35, -120)
BC_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_EB/", "Sat_BC_EB_", 35, -115)
BC_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_EC/", "Sat_BC_EC_", 35, -110)
BC_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_ED/", "Sat_BC_ED_", 35, -105)
BC_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_EE/", "Sat_BC_EE_", 35, -100)
BC_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_EF/", "Sat_BC_EF_", 35,  -95)

BC_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FA/", "Sat_BC_FA_", 30, -120)
BC_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FB/", "Sat_BC_FB_", 30, -115)
BC_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FC/", "Sat_BC_FC_", 30, -110)
BC_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FD/", "Sat_BC_FD_", 30, -105)
BC_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FE/", "Sat_BC_FE_", 30, -100)
BC_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BC_FF/", "Sat_BC_FF_", 30,  -95)

BC_1Degs_List = [
    BC_AA_1Deg, BC_AB_1Deg, BC_AC_1Deg, BC_AD_1Deg, BC_AE_1Deg, BC_AF_1Deg,
    BC_BA_1Deg, BC_BB_1Deg, BC_BC_1Deg, BC_BD_1Deg, BC_BE_1Deg, BC_BF_1Deg,
    BC_CA_1Deg, BC_CB_1Deg, BC_CC_1Deg, BC_CD_1Deg, BC_CE_1Deg, BC_CF_1Deg,
    BC_DA_1Deg, BC_DB_1Deg, BC_DC_1Deg, BC_DD_1Deg, BC_DE_1Deg, BC_DF_1Deg,
    BC_EA_1Deg, BC_EB_1Deg, BC_EC_1Deg, BC_ED_1Deg, BC_EE_1Deg, BC_EF_1Deg,
    BC_FA_1Deg, BC_FB_1Deg, BC_FC_1Deg, BC_FD_1Deg, BC_FE_1Deg, BC_FF_1Deg,
]


In [67]:
# BD - North America - East

BD_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AA/", "Sat_BD_AA_", 55, -90)
BD_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AB/", "Sat_BD_AB_", 55, -85)
BD_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AC/", "Sat_BD_AC_", 55, -80)
BD_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AD/", "Sat_BD_AD_", 55, -75)
BD_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AE/", "Sat_BD_AE_", 55, -70)
BD_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_AF/", "Sat_BD_AF_", 55, -65)

BD_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BA/", "Sat_BD_BA_", 50, -90)
BD_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BB/", "Sat_BD_BB_", 50, -85)
BD_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BC/", "Sat_BD_BC_", 50, -80)
BD_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BD/", "Sat_BD_BD_", 50, -75)
BD_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BE/", "Sat_BD_BE_", 50, -70)
BD_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_BF/", "Sat_BD_BF_", 50, -65)

BD_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CA/", "Sat_BD_CA_", 45, -90)
BD_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CB/", "Sat_BD_CB_", 45, -85)
BD_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CC/", "Sat_BD_CC_", 45, -80)
BD_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CD/", "Sat_BD_CD_", 45, -75)
BD_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CE/", "Sat_BD_CE_", 45, -70)
BD_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_CF/", "Sat_BD_CF_", 45, -65)

BD_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DA/", "Sat_BD_DA_", 40, -90)
BD_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DB/", "Sat_BD_DB_", 40, -85)
BD_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DC/", "Sat_BD_DC_", 40, -80)
BD_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DD/", "Sat_BD_DD_", 40, -75)
BD_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DE/", "Sat_BD_DE_", 40, -70)
BD_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_DF/", "Sat_BD_DF_", 40, -65)

BD_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_EA/", "Sat_BD_EA_", 35, -90)
BD_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_EB/", "Sat_BD_EB_", 35, -85)
BD_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_EC/", "Sat_BD_EC_", 35, -80)
BD_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_ED/", "Sat_BD_ED_", 35, -75)
BD_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_EE/", "Sat_BD_EE_", 35, -70)
BD_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_EF/", "Sat_BD_EF_", 35, -65)

BD_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FA/", "Sat_BD_FA_", 30, -90)
BD_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FB/", "Sat_BD_FB_", 30, -85)
BD_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FC/", "Sat_BD_FC_", 30, -80)
BD_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FD/", "Sat_BD_FD_", 30, -75)
BD_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FE/", "Sat_BD_FE_", 30, -70)
BD_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BD_FF/", "Sat_BD_FF_", 30, -65)

BD_1Degs_List = [
    BD_AA_1Deg, BD_AB_1Deg, BD_AC_1Deg, BD_AD_1Deg, BD_AE_1Deg, BD_AF_1Deg,
    BD_BA_1Deg, BD_BB_1Deg, BD_BC_1Deg, BD_BD_1Deg, BD_BE_1Deg, BD_BF_1Deg,
    BD_CA_1Deg, BD_CB_1Deg, BD_CC_1Deg, BD_CD_1Deg, BD_CE_1Deg, BD_CF_1Deg,
    BD_DA_1Deg, BD_DB_1Deg, BD_DC_1Deg, BD_DD_1Deg, BD_DE_1Deg, BD_DF_1Deg,
    BD_EA_1Deg, BD_EB_1Deg, BD_EC_1Deg, BD_ED_1Deg,
    BD_FA_1Deg, BD_FB_1Deg, BD_FC_1Deg,
]


In [ ]:
# BE - Nova Scotia

BE_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BE_BA/", "Sat_BE_BA_", 50, -60)

BE_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BE_CA/", "Sat_BE_CA_", 45, -60)
BE_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BE_CB/", "Sat_BE_CB_", 45, -55)

BE_1Degs_List = [
    BE_BA_1Deg,
    BE_CA_1Deg, BE_CB_1Deg,
]

In [ ]:
# BF - North Atlantic, UK, Iberia

BF_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AA/", "Sat_BF_AA_", 55, -30)
BF_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AB/", "Sat_BF_AB_", 55, -25)
BF_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AC/", "Sat_BF_AC_", 55, -20)
BF_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AD/", "Sat_BF_AD_", 55, -15)
BF_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AE/", "Sat_BF_AE_", 55, -10)
BF_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_AF/", "Sat_BF_AF_", 55,  -5)

BF_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BA/", "Sat_BF_BA_", 50, -30)
BF_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BB/", "Sat_BF_BB_", 50, -25)
BF_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BC/", "Sat_BF_BC_", 50, -20)
BF_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BD/", "Sat_BF_BD_", 50, -15)
BF_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BE/", "Sat_BF_BE_", 50, -10)
BF_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_BF/", "Sat_BF_BF_", 50,  -5)

BF_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CA/", "Sat_BF_CA_", 45, -30)
BF_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CB/", "Sat_BF_CB_", 45, -25)
BF_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CC/", "Sat_BF_CC_", 45, -20)
BF_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CD/", "Sat_BF_CD_", 45, -15)
BF_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CE/", "Sat_BF_CE_", 45, -10)
BF_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_CF/", "Sat_BF_CF_", 45,  -5)

BF_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DA/", "Sat_BF_DA_", 40, -30)
BF_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DB/", "Sat_BF_DB_", 40, -25)
BF_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DC/", "Sat_BF_DC_", 40, -20)
BF_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DD/", "Sat_BF_DD_", 40, -15)
BF_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DE/", "Sat_BF_DE_", 40, -10)
BF_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_DF/", "Sat_BF_DF_", 40,  -5)

BF_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EA/", "Sat_BF_EA_", 35, -30)
BF_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EB/", "Sat_BF_EB_", 35, -25)
BF_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EC/", "Sat_BF_EC_", 35, -20)
BF_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_ED/", "Sat_BF_ED_", 35, -15)
BF_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EE/", "Sat_BF_EE_", 35, -10)
BF_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_EF/", "Sat_BF_EF_", 35,  -5)

BF_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FA/", "Sat_BF_FA_", 30, -30)
BF_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FB/", "Sat_BF_FB_", 30, -25)
BF_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FC/", "Sat_BF_FC_", 30, -20)
BF_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FD/", "Sat_BF_FD_", 30, -15)
BF_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FE/", "Sat_BF_FE_", 30, -10)
BF_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BF_FF/", "Sat_BF_FF_", 30,  -5)

BF_1Degs_List = [
                BF_AE_1Deg, BF_AF_1Deg,
    BF_BD_1Deg, BF_BE_1Deg, BF_BF_1Deg,
                BF_CE_1Deg, BF_CF_1Deg,
                BF_DE_1Deg, BF_DF_1Deg,
                BF_EE_1Deg, BF_EF_1Deg
]

In [37]:
# BG - Europe

BG_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AA/", "Sat_BG_AA_", 55, 0)
BG_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AB/", "Sat_BG_AB_", 55, 5)
BG_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AC/", "Sat_BG_AC_", 55, 10)
BG_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AD/", "Sat_BG_AD_", 55, 15)
BG_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AE/", "Sat_BG_AE_", 55, 20)
BG_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_AF/", "Sat_BG_AF_", 55, 25)

BG_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BA/", "Sat_BG_BA_", 50, 0)
BG_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BB/", "Sat_BG_BB_", 50, 5)
BG_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BC/", "Sat_BG_BC_", 50, 10)
BG_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BD/", "Sat_BG_BD_", 50, 15)
BG_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BE/", "Sat_BG_BE_", 50, 20)
BG_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_BF/", "Sat_BG_BF_", 50, 25)

BG_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CA/", "Sat_BG_CA_", 45, 0)
BG_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CB/", "Sat_BG_CB_", 45, 5)
BG_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CC/", "Sat_BG_CC_", 45, 10)
BG_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CD/", "Sat_BG_CD_", 45, 15)
BG_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CE/", "Sat_BG_CE_", 45, 20)
BG_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_CF/", "Sat_BG_CF_", 45, 25)

BG_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DA/", "Sat_BG_DA_", 40, 0)
BG_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DB/", "Sat_BG_DB_", 40, 5)
BG_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DC/", "Sat_BG_DC_", 40, 10)
BG_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DD/", "Sat_BG_DD_", 40, 15)
BG_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DE/", "Sat_BG_DE_", 40, 20)
BG_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_DF/", "Sat_BG_DF_", 40, 25)

BG_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EA/", "Sat_BG_EA_", 35, 0)
BG_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EB/", "Sat_BG_EB_", 35, 5)
BG_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EC/", "Sat_BG_EC_", 35, 10)
BG_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_ED/", "Sat_BG_ED_", 35, 15)
BG_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EE/", "Sat_BG_EE_", 35, 20)
BG_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_EF/", "Sat_BG_EF_", 35, 25)

BG_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FA/", "Sat_BG_FA_", 30, 0)
BG_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FB/", "Sat_BG_FB_", 30, 5)
BG_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FC/", "Sat_BG_FC_", 30, 10)
BG_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FD/", "Sat_BG_FD_", 30, 15)
BG_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FE/", "Sat_BG_FE_", 30, 20)
BG_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BG_FF/", "Sat_BG_FF_", 30, 25)

BG_1Degs_List = [
    BG_AA_1Deg, BG_AB_1Deg, BG_AC_1Deg, BG_AD_1Deg, BG_AE_1Deg, BG_AF_1Deg,
    BG_BA_1Deg, BG_BB_1Deg, BG_BC_1Deg, BG_BD_1Deg, BG_BE_1Deg, BG_BF_1Deg,
    BG_CA_1Deg, BG_CB_1Deg, BG_CC_1Deg, BG_CD_1Deg, BG_CE_1Deg, BG_CF_1Deg,
    BG_DA_1Deg, BG_DB_1Deg, BG_DC_1Deg, BG_DD_1Deg, BG_DE_1Deg, BG_DF_1Deg,
    BG_EA_1Deg, BG_EB_1Deg, BG_EC_1Deg, BG_ED_1Deg, BG_EE_1Deg, BG_EF_1Deg,
    BG_FA_1Deg, BG_FB_1Deg, BG_FC_1Deg, BG_FD_1Deg, BG_FE_1Deg, BG_FF_1Deg,
]

BG_1Degs_List = [
    BG_BB_1Deg, BG_BC_1Deg
]

In [16]:
# BH - Black Sea

BH_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AA/", "Sat_BH_AA_", 55, 30)
BH_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AB/", "Sat_BH_AB_", 55, 35)
BH_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AC/", "Sat_BH_AC_", 55, 40)
BH_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AD/", "Sat_BH_AD_", 55, 45)
BH_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AE/", "Sat_BH_AE_", 55, 50)
BH_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_AF/", "Sat_BH_AF_", 55, 55)

BH_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BA/", "Sat_BH_BA_", 50, 30)
BH_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BB/", "Sat_BH_BB_", 50, 35)
BH_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BC/", "Sat_BH_BC_", 50, 40)
BH_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BD/", "Sat_BH_BD_", 50, 45)
BH_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BE/", "Sat_BH_BE_", 50, 50)
BH_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_BF/", "Sat_BH_BF_", 50, 55)

BH_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CA/", "Sat_BH_CA_", 45, 30)
BH_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CB/", "Sat_BH_CB_", 45, 35)
BH_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CC/", "Sat_BH_CC_", 45, 40)
BH_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CD/", "Sat_BH_CD_", 45, 45)
BH_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CE/", "Sat_BH_CE_", 45, 50)
BH_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_CF/", "Sat_BH_CF_", 45, 55)

BH_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DA/", "Sat_BH_DA_", 40, 30)
BH_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DB/", "Sat_BH_DB_", 40, 35)
BH_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DC/", "Sat_BH_DC_", 40, 40)
BH_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DD/", "Sat_BH_DD_", 40, 45)
BH_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DE/", "Sat_BH_DE_", 40, 50)
BH_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_DF/", "Sat_BH_DF_", 40, 55)

BH_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EA/", "Sat_BH_EA_", 35, 30)
BH_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EB/", "Sat_BH_EB_", 35, 35)
BH_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EC/", "Sat_BH_EC_", 35, 40)
BH_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_ED/", "Sat_BH_ED_", 35, 45)
BH_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EE/", "Sat_BH_EE_", 35, 50)
BH_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_EF/", "Sat_BH_EF_", 35, 55)

BH_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FA/", "Sat_BH_FA_", 30, 30)
BH_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FB/", "Sat_BH_FB_", 30, 35)
BH_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FC/", "Sat_BH_FC_", 30, 40)
BH_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FD/", "Sat_BH_FD_", 30, 45)
BH_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FE/", "Sat_BH_FE_", 30, 50)
BH_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BH_FF/", "Sat_BH_FF_", 30, 55)

BH_1Degs_List = [
    BH_AA_1Deg, BH_AB_1Deg, BH_AC_1Deg, BH_AD_1Deg, BH_AE_1Deg, BH_AF_1Deg,
    BH_BA_1Deg, BH_BB_1Deg, BH_BC_1Deg, BH_BD_1Deg, BH_BE_1Deg, BH_BF_1Deg,
    BH_CA_1Deg, BH_CB_1Deg, BH_CC_1Deg, BH_CD_1Deg, BH_CE_1Deg, BH_CF_1Deg,
    BH_DA_1Deg, BH_DB_1Deg, BH_DC_1Deg, BH_DD_1Deg, BH_DE_1Deg, BH_DF_1Deg,
    BH_EA_1Deg, BH_EB_1Deg, BH_EC_1Deg, BH_ED_1Deg, BH_EE_1Deg, BH_EF_1Deg,
    BH_FA_1Deg, BH_FB_1Deg, BH_FC_1Deg, BH_FD_1Deg, BH_FE_1Deg, BH_FF_1Deg,
]


In [ ]:
# BI - Central Asia

BI_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AA/", "Sat_BI_AA_", 55, 60)
BI_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AB/", "Sat_BI_AB_", 55, 65)
BI_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AC/", "Sat_BI_AC_", 55, 70)
BI_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AD/", "Sat_BI_AD_", 55, 75)
BI_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AE/", "Sat_BI_AE_", 55, 80)
BI_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_AF/", "Sat_BI_AF_", 55, 85)

BI_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BA/", "Sat_BI_BA_", 50, 60)
BI_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BB/", "Sat_BI_BB_", 50, 65)
BI_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BC/", "Sat_BI_BC_", 50, 70)
BI_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BD/", "Sat_BI_BD_", 50, 75)
BI_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BE/", "Sat_BI_BE_", 50, 80)
BI_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_BF/", "Sat_BI_BF_", 50, 85)

BI_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CA/", "Sat_BI_CA_", 45, 60)
BI_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CB/", "Sat_BI_CB_", 45, 65)
BI_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CC/", "Sat_BI_CC_", 45, 70)
BI_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CD/", "Sat_BI_CD_", 45, 75)
BI_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CE/", "Sat_BI_CE_", 45, 80)
BI_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_CF/", "Sat_BI_CF_", 45, 85)

BI_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DA/", "Sat_BI_DA_", 40, 60)
BI_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DB/", "Sat_BI_DB_", 40, 65)
BI_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DC/", "Sat_BI_DC_", 40, 70)
BI_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DD/", "Sat_BI_DD_", 40, 75)
BI_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DE/", "Sat_BI_DE_", 40, 80)
BI_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_DF/", "Sat_BI_DF_", 40, 85)

BI_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_EA/", "Sat_BI_EA_", 35, 60)
BI_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_EB/", "Sat_BI_EB_", 35, 65)
BI_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_EC/", "Sat_BI_EC_", 35, 70)
BI_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_ED/", "Sat_BI_ED_", 35, 75)
BI_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_EE/", "Sat_BI_EE_", 35, 80)
BI_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_EF/", "Sat_BI_EF_", 35, 85)

BI_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FA/", "Sat_BI_FA_", 30, 60)
BI_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FB/", "Sat_BI_FB_", 30, 65)
BI_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FC/", "Sat_BI_FC_", 30, 70)
BI_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FD/", "Sat_BI_FD_", 30, 75)
BI_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FE/", "Sat_BI_FE_", 30, 80)
BI_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BI_FF/", "Sat_BI_FF_", 30, 85)

BI_1Degs_List = [
    BI_AA_1Deg, BI_AB_1Deg, BI_AC_1Deg, BI_AD_1Deg, BI_AE_1Deg, BI_AF_1Deg,
    BI_BA_1Deg, BI_BB_1Deg, BI_BC_1Deg, BI_BD_1Deg, BI_BE_1Deg, BI_BF_1Deg,
    BI_CA_1Deg, BI_CB_1Deg, BI_CC_1Deg, BI_CD_1Deg, BI_CE_1Deg, BI_CF_1Deg,
    BI_DA_1Deg, BI_DB_1Deg, BI_DC_1Deg, BI_DD_1Deg, BI_DE_1Deg, BI_DF_1Deg,
    BI_EA_1Deg, BI_EB_1Deg, BI_EC_1Deg, BI_ED_1Deg, BI_EE_1Deg, BI_EF_1Deg,
    BI_FA_1Deg, BI_FB_1Deg, BI_FC_1Deg, BI_FD_1Deg, BI_FE_1Deg, BI_FF_1Deg,
]


In [ ]:
# BJ - Central Asia

BJ_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AA/", "Sat_BJ_AA_", 55, 90)
BJ_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AB/", "Sat_BJ_AB_", 55, 95)
BJ_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AC/", "Sat_BJ_AC_", 55, 100)
BJ_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AD/", "Sat_BJ_AD_", 55, 105)
BJ_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AE/", "Sat_BJ_AE_", 55, 110)
BJ_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_AF/", "Sat_BJ_AF_", 55, 115)

BJ_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BA/", "Sat_BJ_BA_", 50, 90)
BJ_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BB/", "Sat_BJ_BB_", 50, 95)
BJ_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BC/", "Sat_BJ_BC_", 50, 100)
BJ_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BD/", "Sat_BJ_BD_", 50, 105)
BJ_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BE/", "Sat_BJ_BE_", 50, 110)
BJ_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_BF/", "Sat_BJ_BF_", 50, 115)

BJ_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CA/", "Sat_BJ_CA_", 45, 90)
BJ_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CB/", "Sat_BJ_CB_", 45, 95)
BJ_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CC/", "Sat_BJ_CC_", 45, 100)
BJ_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CD/", "Sat_BJ_CD_", 45, 105)
BJ_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CE/", "Sat_BJ_CE_", 45, 110)
BJ_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_CF/", "Sat_BJ_CF_", 45, 115)

BJ_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DA/", "Sat_BJ_DA_", 40, 90)
BJ_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DB/", "Sat_BJ_DB_", 40, 95)
BJ_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DC/", "Sat_BJ_DC_", 40, 100)
BJ_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DD/", "Sat_BJ_DD_", 40, 105)
BJ_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DE/", "Sat_BJ_DE_", 40, 110)
BJ_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_DF/", "Sat_BJ_DF_", 40, 115)

BJ_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_EA/", "Sat_BJ_EA_", 35, 90)
BJ_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_EB/", "Sat_BJ_EB_", 35, 95)
BJ_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_EC/", "Sat_BJ_EC_", 35, 100)
BJ_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_ED/", "Sat_BJ_ED_", 35, 105)
BJ_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_EE/", "Sat_BJ_EE_", 35, 110)
BJ_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_EF/", "Sat_BJ_EF_", 35, 115)

BJ_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FA/", "Sat_BJ_FA_", 30, 90)
BJ_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FB/", "Sat_BJ_FB_", 30, 95)
BJ_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FC/", "Sat_BJ_FC_", 30, 100)
BJ_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FD/", "Sat_BJ_FD_", 30, 105)
BJ_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FE/", "Sat_BJ_FE_", 30, 110)
BJ_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BJ_FF/", "Sat_BJ_FF_", 30, 115)

BJ_1Degs_List = [
    BJ_AA_1Deg, BJ_AB_1Deg, BJ_AC_1Deg, BJ_AD_1Deg, BJ_AE_1Deg, BJ_AF_1Deg,
    BJ_BA_1Deg, BJ_BB_1Deg, BJ_BC_1Deg, BJ_BD_1Deg, BJ_BE_1Deg, BJ_BF_1Deg,
    BJ_CA_1Deg, BJ_CB_1Deg, BJ_CC_1Deg, BJ_CD_1Deg, BJ_CE_1Deg, BJ_CF_1Deg,
    BJ_DA_1Deg, BJ_DB_1Deg, BJ_DC_1Deg, BJ_DD_1Deg, BJ_DE_1Deg, BJ_DF_1Deg,
    BJ_EA_1Deg, BJ_EB_1Deg, BJ_EC_1Deg, BJ_ED_1Deg, BJ_EE_1Deg, BJ_EF_1Deg,
    BJ_FA_1Deg, BJ_FB_1Deg, BJ_FC_1Deg, BJ_FD_1Deg, BJ_FE_1Deg, BJ_FF_1Deg,
]


In [ ]:
# BK - Korea and Japan

BK_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AA/", "Sat_BK_AA_", 55, 120)
BK_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AB/", "Sat_BK_AB_", 55, 125)
BK_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AC/", "Sat_BK_AC_", 55, 130)
BK_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AD/", "Sat_BK_AD_", 55, 135)
BK_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AE/", "Sat_BK_AE_", 55, 140)
BK_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_AF/", "Sat_BK_AF_", 55, 145)

BK_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BA/", "Sat_BK_BA_", 50, 120)
BK_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BB/", "Sat_BK_BB_", 50, 125)
BK_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BC/", "Sat_BK_BC_", 50, 130)
BK_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BD/", "Sat_BK_BD_", 50, 135)
BK_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BE/", "Sat_BK_BE_", 50, 140)
BK_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_BF/", "Sat_BK_BF_", 50, 145)

BK_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CA/", "Sat_BK_CA_", 45, 120)
BK_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CB/", "Sat_BK_CB_", 45, 125)
BK_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CC/", "Sat_BK_CC_", 45, 130)
BK_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CD/", "Sat_BK_CD_", 45, 135)
BK_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CE/", "Sat_BK_CE_", 45, 140)
BK_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_CF/", "Sat_BK_CF_", 45, 145)

BK_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DA/", "Sat_BK_DA_", 40, 120)
BK_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DB/", "Sat_BK_DB_", 40, 125)
BK_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DC/", "Sat_BK_DC_", 40, 130)
BK_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DD/", "Sat_BK_DD_", 40, 135)
BK_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DE/", "Sat_BK_DE_", 40, 140)
BK_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_DF/", "Sat_BK_DF_", 40, 145)

BK_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_EA/", "Sat_BK_EA_", 35, 120)
BK_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_EB/", "Sat_BK_EB_", 35, 125)
BK_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_EC/", "Sat_BK_EC_", 35, 130)
BK_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_ED/", "Sat_BK_ED_", 35, 135)
BK_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_EE/", "Sat_BK_EE_", 35, 140)
BK_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_EF/", "Sat_BK_EF_", 35, 145)

BK_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FA/", "Sat_BK_FA_", 30, 120)
BK_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FB/", "Sat_BK_FB_", 30, 125)
BK_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FC/", "Sat_BK_FC_", 30, 130)
BK_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FD/", "Sat_BK_FD_", 30, 135)
BK_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FE/", "Sat_BK_FE_", 30, 140)
BK_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/BK_FF/", "Sat_BK_FF_", 30, 145)

BK_1Degs_List = [
    BK_AA_1Deg, BK_AB_1Deg, BK_AC_1Deg, BK_AD_1Deg, BK_AE_1Deg, BK_AF_1Deg,
    BK_BA_1Deg, BK_BB_1Deg, BK_BC_1Deg, BK_BD_1Deg, BK_BE_1Deg, BK_BF_1Deg,
    BK_CA_1Deg, BK_CB_1Deg, BK_CC_1Deg, BK_CD_1Deg, BK_CE_1Deg, BK_CF_1Deg,
    BK_DA_1Deg, BK_DB_1Deg, BK_DC_1Deg, BK_DD_1Deg, BK_DE_1Deg, BK_DF_1Deg,
    BK_EA_1Deg, BK_EB_1Deg, BK_EC_1Deg, BK_ED_1Deg, BK_EE_1Deg, BK_EF_1Deg,
    BK_FA_1Deg, BK_FB_1Deg, BK_FC_1Deg, BK_FD_1Deg, BK_FE_1Deg, BK_FF_1Deg,
]

In [ ]:
# BL - Empty Pacific Ocean

In [ ]:
# CA - Hawaii and Pacific Ocean

CA_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/CA_BE/", "Sat_CA_BE_", 20, 160)

CA_1Degs_List = [
    CA_BE_1Deg
]

In [ ]:
# CB - Empty Pacific Ocean

In [55]:
# CC - Mexico

CC_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AA/", "Sat_CC_AA_", 25, -120)
CC_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AB/", "Sat_CC_AB_", 25, -115)
CC_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AC/", "Sat_CC_AC_", 25, -110)
CC_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AD/", "Sat_CC_AD_", 25, -105)
CC_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AE/", "Sat_CC_AE_", 25, -100)
CC_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_AF/", "Sat_CC_AF_", 25,  -95)

CC_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BA/", "Sat_CC_BA_", 20, -120)
CC_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BB/", "Sat_CC_BB_", 20, -115)
CC_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BC/", "Sat_CC_BC_", 20, -110)
CC_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BD/", "Sat_CC_BD_", 20, -105)
CC_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BE/", "Sat_CC_BE_", 20, -100)
CC_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_BF/", "Sat_CC_BF_", 20,  -95)

CC_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CA/", "Sat_CC_CA_", 15, -120)
CC_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CB/", "Sat_CC_CB_", 15, -115)
CC_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CC/", "Sat_CC_CC_", 15, -110)
CC_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CD/", "Sat_CC_CD_", 15, -105)
CC_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CE/", "Sat_CC_CE_", 15, -100)
CC_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_CF/", "Sat_CC_CF_", 15,  -95)

CC_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DA/", "Sat_CC_DA_", 10, -120)
CC_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DB/", "Sat_CC_DB_", 10, -115)
CC_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DC/", "Sat_CC_DC_", 10, -110)
CC_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DD/", "Sat_CC_DD_", 10, -105)
CC_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DE/", "Sat_CC_DE_", 10, -100)
CC_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_DF/", "Sat_CC_DF_", 10,  -95)

CC_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EA/", "Sat_CC_EA_",  5, -120)
CC_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EB/", "Sat_CC_EB_",  5, -115)
CC_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EC/", "Sat_CC_EC_",  5, -110)
CC_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_ED/", "Sat_CC_ED_",  5, -105)
CC_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EE/", "Sat_CC_EE_",  5, -100)
CC_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EF/", "Sat_CC_EF_",  5,  -95)

CC_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FA/", "Sat_CC_FA_",  0, -120)
CC_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FB/", "Sat_CC_FB_",  0, -115)
CC_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FC/", "Sat_CC_FC_",  0, -110)
CC_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FD/", "Sat_CC_FD_",  0, -105)
CC_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FE/", "Sat_CC_FE_",  0, -100)
CC_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FF/", "Sat_CC_FF_",  0,  -95)

CC_1Degs_List = [
    CC_AA_1Deg, CC_AB_1Deg, CC_AC_1Deg, CC_AD_1Deg, CC_AE_1Deg, CC_AF_1Deg,
                CC_BB_1Deg, CC_BC_1Deg, CC_BD_1Deg, CC_BE_1Deg, CC_BF_1Deg,
                            CC_CC_1Deg, CC_CD_1Deg, CC_CE_1Deg, CC_CF_1Deg,
                                        CC_DD_1Deg, CC_DE_1Deg, CC_DF_1Deg,

]

In [56]:
# CD - Cuba

CD_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AA/", "Sat_CD_AA_", 25, -90)
CD_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AB/", "Sat_CD_AB_", 25, -85)
CD_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AC/", "Sat_CD_AC_", 25, -80)
CD_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AD/", "Sat_CD_AD_", 25, -75)
CD_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AE/", "Sat_CD_AE_", 25, -70)
CD_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_AF/", "Sat_CD_AF_", 25, -65)

CD_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BA/", "Sat_CD_BA_", 20, -90)
CD_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BB/", "Sat_CD_BB_", 20, -85)
CD_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BC/", "Sat_CD_BC_", 20, -80)
CD_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BD/", "Sat_CD_BD_", 20, -75)
CD_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BE/", "Sat_CD_BE_", 20, -70)
CD_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_BF/", "Sat_CD_BF_", 20, -65)

CD_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CA/", "Sat_CD_CA_", 15, -90)
CD_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CB/", "Sat_CD_CB_", 15, -85)
CD_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CC/", "Sat_CD_CC_", 15, -80)
CD_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CD/", "Sat_CD_CD_", 15, -75)
CD_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CE/", "Sat_CD_CE_", 15, -70)
CD_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_CF/", "Sat_CD_CF_", 15, -65)

CD_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DA/", "Sat_CD_DA_", 10, -90)
CD_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DB/", "Sat_CD_DB_", 10, -85)
CD_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DC/", "Sat_CD_DC_", 10, -80)
CD_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DD/", "Sat_CD_DD_", 10, -75)
CD_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DE/", "Sat_CD_DE_", 10, -70)
CD_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_DF/", "Sat_CD_DF_", 10, -65)

CD_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_EA/", "Sat_CD_EA_",  5, -90)
CD_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_EB/", "Sat_CD_EB_",  5, -85)
CD_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_EC/", "Sat_CD_EC_",  5, -80)
CD_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_ED/", "Sat_CD_ED_",  5, -75)
CD_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_EE/", "Sat_CD_EE_",  5, -70)
CD_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_EF/", "Sat_CD_EF_",  5, -65)

CD_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FA/", "Sat_CD_FA_",  0, -90)
CD_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FB/", "Sat_CD_FB_",  0, -85)
CD_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FC/", "Sat_CD_FC_",  0, -80)
CD_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FD/", "Sat_CD_FD_",  0, -75)
CD_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FE/", "Sat_CD_FE_",  0, -70)
CD_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/CD_FF/", "Sat_CD_FF_",  0, -65)

CD_1Degs_List = [
    CD_AA_1Deg, CD_AB_1Deg, CD_AC_1Deg,
    CD_BA_1Deg, CD_BB_1Deg, CD_BC_1Deg, CD_BD_1Deg,
    CD_CA_1Deg, CD_CB_1Deg, CD_CC_1Deg, CD_CD_1Deg, CD_CE_1Deg, CD_CF_1Deg,
    CD_DA_1Deg, CD_DB_1Deg, CD_DC_1Deg, CD_DD_1Deg, CD_DE_1Deg, CD_DF_1Deg,
    CD_EA_1Deg, CD_EB_1Deg, CD_EC_1Deg, CD_ED_1Deg, CD_EE_1Deg, CD_EF_1Deg,
                            CD_FC_1Deg, CD_FD_1Deg, CD_FE_1Deg, CD_FF_1Deg,
]

# Loop through the dictionaries, and then fields
for dictionary in CD_1Degs_List:
    for key, value in dictionary.items():
        print(f"\"{key}\": {value},")

"Sat_CD_AA_AA.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_AA.png', 'lat': 29, 'long': -90, 'width': 1},
"Sat_CD_AA_AB.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_AB.png', 'lat': 29, 'long': -89, 'width': 1},
"Sat_CD_AA_AC.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_AC.png', 'lat': 29, 'long': -88, 'width': 1},
"Sat_CD_AA_AD.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_AD.png', 'lat': 29, 'long': -87, 'width': 1},
"Sat_CD_AA_AE.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_AE.png', 'lat': 29, 'long': -86, 'width': 1},
"Sat_CD_AA_BA.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_BA.png', 'lat': 28, 'long': -90, 'width': 1},
"Sat_CD_AA_BB.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_BB.png', 'lat': 28, 'long': -89, 'width': 1},
"Sat_CD_AA_BC.png": {'localpath': 'Lvl2_1x1/CD_AA/', 'filename': 'Sat_CD_AA_BC.png', 'lat': 28, 'long': -88, 'width': 1},
"Sat_CD_AA_BD.png": {'lo

In [ ]:
# CE - Guyana & Empty Atlantic Ocean

CE_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EA/", "Sat_CE_EA_", 5, -60)
CE_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_EB/", "Sat_CE_EB_", 5, -55)

CE_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FA/", "Sat_CE_FA_", 0, -60)
CE_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FB/", "Sat_CE_FB_", 0, -55)
CE_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/CC_FC/", "Sat_CE_FC_", 0, -50)

CE_1Degs_List = [
    CE_EA_1Deg, CE_EB_1Deg,
    CE_FA_1Deg, CE_FB_1Deg, CE_FC_1Deg,
]

In [ ]:
# CF - West Africa

CF_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AA/", "Sat_CF_AA_", 25, -30)
CF_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AB/", "Sat_CF_AB_", 25, -25)
CF_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AC/", "Sat_CF_AC_", 25, -20)
CF_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AD/", "Sat_CF_AD_", 25, -15)
CF_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AE/", "Sat_CF_AE_", 25, -10)
CF_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_AF/", "Sat_CF_AF_", 25,  -5)

CF_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BA/", "Sat_CF_BA_", 20, -30)
CF_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BB/", "Sat_CF_BB_", 20, -25)
CF_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BC/", "Sat_CF_BC_", 20, -20)
CF_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BD/", "Sat_CF_BD_", 20, -15)
CF_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BE/", "Sat_CF_BE_", 20, -10)
CF_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_BF/", "Sat_CF_BF_", 20,  -5)

CF_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CA/", "Sat_CF_CA_", 15, -30)
CF_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CB/", "Sat_CF_CB_", 15, -25)
CF_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CC/", "Sat_CF_CC_", 15, -20)
CF_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CD/", "Sat_CF_CD_", 15, -15)
CF_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CE/", "Sat_CF_CE_", 15, -10)
CF_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_CF/", "Sat_CF_CF_", 15,  -5)

CF_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DA/", "Sat_CF_DA_", 10, -30)
CF_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DB/", "Sat_CF_DB_", 10, -25)
CF_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DC/", "Sat_CF_DC_", 10, -20)
CF_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DD/", "Sat_CF_DD_", 10, -15)
CF_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DE/", "Sat_CF_DE_", 10, -10)
CF_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_DF/", "Sat_CF_DF_", 10,  -5)

CF_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_EA/", "Sat_CF_EA_",  5, -30)
CF_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_EB/", "Sat_CF_EB_",  5, -25)
CF_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_EC/", "Sat_CF_EC_",  5, -20)
CF_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_ED/", "Sat_CF_ED_",  5, -15)
CF_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_EE/", "Sat_CF_EE_",  5, -10)
CF_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_EF/", "Sat_CF_EF_",  5,  -5)

CF_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FA/", "Sat_CF_FA_",  0, -30)
CF_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FB/", "Sat_CF_FB_",  0, -25)
CF_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FC/", "Sat_CF_FC_",  0, -20)
CF_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FD/", "Sat_CF_FD_",  0, -15)
CF_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FE/", "Sat_CF_FE_",  0, -10)
CF_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/CF_FF/", "Sat_CF_FF_",  0,  -5)

CF_1Degs_List = [
    CF_AA_1Deg, CF_AB_1Deg, CF_AC_1Deg, CF_AD_1Deg, CF_AE_1Deg, CF_AF_1Deg,
    CF_BA_1Deg, CF_BB_1Deg, CF_BC_1Deg, CF_BD_1Deg, CF_BE_1Deg, CF_BF_1Deg,
    CF_CA_1Deg, CF_CB_1Deg, CF_CC_1Deg, CF_CD_1Deg, CF_CE_1Deg, CF_CF_1Deg,
    CF_DA_1Deg, CF_DB_1Deg, CF_DC_1Deg, CF_DD_1Deg, CF_DE_1Deg, CF_DF_1Deg,
    CF_EA_1Deg, CF_EB_1Deg, CF_EC_1Deg, CF_ED_1Deg, CF_EE_1Deg, CF_EF_1Deg,
    CF_FA_1Deg, CF_FB_1Deg, CF_FC_1Deg, CF_FD_1Deg, CF_FE_1Deg, CF_FF_1Deg,
]


In [ ]:
# CG -Central Africa

CG_AA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AA/", "Sat_CG_AA_", 25,  0)
CG_AB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AB/", "Sat_CG_AB_", 25,  5)
CG_AC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AC/", "Sat_CG_AC_", 25, 10)
CG_AD_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AD/", "Sat_CG_AD_", 25, 15)
CG_AE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AE/", "Sat_CG_AE_", 25, 20)
CG_AF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_AF/", "Sat_CG_AF_", 25, 25)

CG_BA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BA/", "Sat_CG_BA_", 20,  0)
CG_BB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BB/", "Sat_CG_BB_", 20,  5)
CG_BC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BC/", "Sat_CG_BC_", 20, 10)
CG_BD_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BD/", "Sat_CG_BD_", 20, 15)
CG_BE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BE/", "Sat_CG_BE_", 20, 20)
CG_BF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_BF/", "Sat_CG_BF_", 20, 25)

CG_CA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CA/", "Sat_CG_CA_", 15,  0)
CG_CB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CB/", "Sat_CG_CB_", 15,  5)
CG_CC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CC/", "Sat_CG_CC_", 15, 10)
CG_CD_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CD/", "Sat_CG_CD_", 15, 15)
CG_CE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CE/", "Sat_CG_CE_", 15, 20)
CG_CF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_CF/", "Sat_CG_CF_", 15, 25)

CG_DA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DA/", "Sat_CG_DA_", 10,  0)
CG_DB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DB/", "Sat_CG_DB_", 10,  5)
CG_DC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DC/", "Sat_CG_DC_", 10, 10)
CG_DD_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DD/", "Sat_CG_DD_", 10, 15)
CG_DE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DE/", "Sat_CG_DE_", 10, 20)
CG_DF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_DF/", "Sat_CG_DF_", 10, 25)

CG_EA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_EA/", "Sat_CG_EA_",  5,  0)
CG_EB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_EB/", "Sat_CG_EB_",  5,  5)
CG_EC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_EC/", "Sat_CG_EC_",  5, 10)
CG_ED_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_ED/", "Sat_CG_ED_",  5, 15)
CG_EE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_EE/", "Sat_CG_EE_",  5, 20)
CG_EF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_EF/", "Sat_CG_EF_",  5, 25)

CG_FA_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FA/", "Sat_CG_FA_",  0,  0)
CG_FB_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FB/", "Sat_CG_FB_",  0,  5)
CG_FC_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FC/", "Sat_CG_FC_",  0, 10)
CG_FD_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FD/", "Sat_CG_FD_",  0, 15)
CG_FE_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FE/", "Sat_CG_FE_",  0, 20)
CG_FF_1Deg = create_5deg_dictionary("Lvl2_1x1/CG_FF/", "Sat_CG_FF_",  0, 25)

CG_1Degs_List = [
    CG_AA_1Deg, CG_AB_1Deg, CG_AC_1Deg, CG_AD_1Deg, CG_AE_1Deg, CG_AF_1Deg,
    CG_BA_1Deg, CG_BB_1Deg, CG_BC_1Deg, CG_BD_1Deg, CG_BE_1Deg, CG_BF_1Deg,
    CG_CA_1Deg, CG_CB_1Deg, CG_CC_1Deg, CG_CD_1Deg, CG_CE_1Deg, CG_CF_1Deg,
    CG_DA_1Deg, CG_DB_1Deg, CG_DC_1Deg, CG_DD_1Deg, CG_DE_1Deg, CG_DF_1Deg,
    CG_EA_1Deg, CG_EB_1Deg, CG_EC_1Deg, CG_ED_1Deg, CG_EE_1Deg, CG_EF_1Deg,
    CG_FA_1Deg, CG_FB_1Deg, CG_FC_1Deg, CG_FD_1Deg, CG_FE_1Deg, CG_FF_1Deg,
]


In [45]:
# Hamburg - Sub 1Deg - BH_BB_BE / BG_BC_BA

BG_BB_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BB_BD/", "Sat_BG_BB_BD_",  53,  8)
BG_BB_BE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BB_BE/", "Sat_BG_BB_BE_",  53,  9)
BG_BC_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_BC_BA/", "Sat_BG_BC_BA_",  53, 10)

Hamburg_Sub1Degs_List = [
    BG_BB_BD_Sub1Deg,
    BG_BB_BE_Sub1Deg,
    BG_BC_BA_Sub1Deg,
]

# Loop through the dictionaries, and then fields
for dictionary in Hamburg_Sub1Degs_List:
    for key, value in dictionary.items():
        print(f"\"{key}\": {value},")

"Sat_BG_BB_BD_AA.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AA.png', 'lat': 53.8, 'long': 8.0, 'width': 0.2},
"Sat_BG_BB_BD_AB.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AB.png', 'lat': 53.8, 'long': 8.2, 'width': 0.2},
"Sat_BG_BB_BD_AC.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AC.png', 'lat': 53.8, 'long': 8.4, 'width': 0.2},
"Sat_BG_BB_BD_AD.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AD.png', 'lat': 53.8, 'long': 8.6, 'width': 0.2},
"Sat_BG_BB_BD_AE.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_AE.png', 'lat': 53.8, 'long': 8.8, 'width': 0.2},
"Sat_BG_BB_BD_BA.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_BA.png', 'lat': 53.6, 'long': 8.0, 'width': 0.2},
"Sat_BG_BB_BD_BB.png": {'localpath': 'Lvl3_0p2x0p2/BG_BB_BD/', 'filename': 'Sat_BG_BB_BD_BB.png', 'lat': 53.6, 'long': 8.2, 'width': 0.2},
"Sat_BG_BB_BD_BC.png": {'lo

In [ ]:

# Rome - Sub 1Deg

BG_DC_BA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BA/", "Sat_BG_DC_BA_",  43, 10)
BG_DC_BB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BB/", "Sat_BG_DC_BB_",  43, 11)
BG_DC_BC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BC/", "Sat_BG_DC_BC_",  43, 12)
BG_DC_BD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_BD/", "Sat_BG_DC_BD_",  43, 13)

BG_DC_CA_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CA/", "Sat_BG_DC_CA_",  42, 10)
BG_DC_CB_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CB/", "Sat_BG_DC_CB_",  42, 11)
BG_DC_CC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CC/", "Sat_BG_DC_CC_",  42, 12)
BG_DC_CD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CD/", "Sat_BG_DC_CD_",  42, 13)
BG_DC_CE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_CE/", "Sat_BG_DC_CE_",  42, 14)

BG_DC_DC_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DC/", "Sat_BG_DC_DC_",  41, 12)
BG_DC_DD_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DD/", "Sat_BG_DC_DD_",  41, 13)
BG_DC_DE_Sub1Deg = create_1deg_dictionary("Lvl3_0p2x0p2/BG_DC_DE/", "Sat_BG_DC_DE_",  41, 14)

Rome_Sub1Degs_List = [
    BG_DC_BA_Sub1Deg, BG_DC_BB_Sub1Deg, BG_DC_BC_Sub1Deg, BG_DC_BD_Sub1Deg,
    BG_DC_CA_Sub1Deg, BG_DC_CB_Sub1Deg, BG_DC_CC_Sub1Deg, BG_DC_CD_Sub1Deg, BG_DC_CE_Sub1Deg,
                                        BG_DC_DC_Sub1Deg, BG_DC_DD_Sub1Deg, BG_DC_DE_Sub1Deg,
]

# Loop through the dictionaries, and then fields
for dictionary in Rome_Sub1Degs_List:
    for key, value in dictionary.items():
        print(f"\"{key}\": {value},")

# Rome Details
BG_DC_DC_AB_0p2Deg = create_0p2deg_dictionary("Lvl4_0p2x0p2/BG_DC_DC_AB/", "Sat_BG_DC_DC_AB_", 41.80, 12.20)

Rome_0p2Degs_List = [
    BG_DC_DC_AB_0p2Deg,
]

"Sat_BG_DC_BA_AA.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AA.png', 'lat': 43.8, 'long': 10.0, 'width': 0.2},
"Sat_BG_DC_BA_AB.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AB.png', 'lat': 43.8, 'long': 10.2, 'width': 0.2},
"Sat_BG_DC_BA_AC.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AC.png', 'lat': 43.8, 'long': 10.4, 'width': 0.2},
"Sat_BG_DC_BA_AD.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AD.png', 'lat': 43.8, 'long': 10.6, 'width': 0.2},
"Sat_BG_DC_BA_AE.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_AE.png', 'lat': 43.8, 'long': 10.8, 'width': 0.2},
"Sat_BG_DC_BA_BA.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_BA.png', 'lat': 43.6, 'long': 10.0, 'width': 0.2},
"Sat_BG_DC_BA_BB.png": {'localpath': 'Lvl3_0p2x0p2/BG_DC_BA/', 'filename': 'Sat_BG_DC_BA_BB.png', 'lat': 43.6, 'long': 10.2, 'width': 0.2},
"Sat_BG_DC_BA_BC.png

In [24]:

def process_tileset_dict(tileset_dict, datestr):

    for data in tileset_dict.values():
        TilesetDir = os.path.join(root_dir, data["localpath"])
        filepath   = os.path.join(TilesetDir, data["filename"])

        ensure_dir(TilesetDir)

        print(filepath)

        if not os.path.exists(filepath):

            print(f"Downloading and enhancing image: {data['filename']}")

            # Call your functions using the metadata
            download_GM2(filepath, datestr, data["lat"], data["long"], data["width"])


In [34]:
# Loop over the dictionaries and call the download function for each

datestr = globalmosaic_dates["2024Q2"]
brightness = 35

root_dir = f"C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/{datestr}/"
ensure_dir(root_dir)

for tileset_dict in Rome_0p2Degs_List:
    process_tileset_dict(tileset_dict, datestr)

C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_AA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_AB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_AC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_AD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_AE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_BA.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_BB.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_BC.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_BD.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/Sat_BG_DC_DC_AB_BE.png
C:/Util/GlobeLibrary_MapPrep/_WIP/exp/MapLib/Lvl4_0p2x0p2/BG_DC_DC_AB/